In [89]:
import torch
import torch.nn as nn 
from torch import functional as F
import matplotlib.pyplot as plt


In [72]:
# hyperparameters
batch_size = 32
block_size = 128
max_iters = 10000
eval_interval = 500
learning_rate = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
eval_iters = 2000
n_embd = 32


In [73]:

with open('tinyshakespeare.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Split in to train anv validation
data = torch.tensor(encode(text), dtype = torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data= data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x, y

In [91]:

class Transformer(nn.Module): # not actually a bigram model anymore
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) 
        self.sa_heads = MultiHeadAttention(n_heads=4, head_size=n_embd//4)
        self.lm_head = nn.Linear(n_embd, vocab_size) #lm head is 
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensors. where B is batch size and T is the number of tokens in each sequence (block_size*batch_size)

        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.sa_heads(x) #apply one head of self attention, (B,T,C)
        logits = self.lm_head(x)  # (B,T,Vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            #crop idx
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx
    
class Head(nn.Module):
    '''a single self attention head'''
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)
        wei = q @ k.transpose(-2,-1) * (C ** -0.5) # (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        v = self.value(x)
        out = wei @ v
        return out
    

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)
    


In [75]:
m = Transformer()
print(f'Number of parameters: {sum(p.numel() for p in m.parameters())}')    

Number of parameters: 11393


In [85]:
# Training loop
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)
lossi = []
for steps in range(max_iters):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    lossi.append(loss.log10().item())
    if steps % eval_interval == 0:
        print(f'step: {steps}, loss: {loss.item()}')

plt.plot(lossi)
lossi_means = [sum(lossi[i:i+100])/100 for i in range(len(lossi)-100)]
plt.plot(lossi_means)
plt.show()


step: 0, loss: 2.2071707248687744
step: 500, loss: 2.1520121097564697
step: 1000, loss: 2.142669200897217
step: 1500, loss: 2.16644549369812
step: 2000, loss: 2.121833562850952
step: 2500, loss: 2.1706511974334717
step: 3000, loss: 2.1305081844329834
step: 3500, loss: 2.1475746631622314
step: 4000, loss: 2.150425910949707
step: 4500, loss: 2.118091106414795
step: 5000, loss: 2.114518165588379
step: 5500, loss: 2.085477113723755
step: 6000, loss: 2.1351451873779297
step: 6500, loss: 2.100278615951538
step: 7000, loss: 2.086372137069702
step: 7500, loss: 2.104663133621216
step: 8000, loss: 2.0902113914489746
step: 8500, loss: 2.0724220275878906
step: 9000, loss: 2.0806002616882324
step: 9500, loss: 2.0704681873321533


In [92]:
# Evaluation loop over validation set

with torch.no_grad():
    total_loss = 0
    for steps in range(eval_iters):
        xb, yb = get_batch('val')
        logits, loss = m(xb, yb)
        total_loss += loss.item()
    print(f'validation loss: {total_loss/eval_iters}')


validation loss: 2.187486908793449


In [103]:
# Generate some text
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, 200)[0].tolist()))



Engy min the; pred astos times mee have your ther tred,
Thown off
bith tof, it the it dathe hiret.

Hist hit
Thaves eave tre stom ansten for gral':
Who nor, by ulasttly to he ther or of I sand of nat 
